In [3]:
using Distributions
using Images

@everywhere function compute_filters(img,ground_truth_im)
    w,h = size(img)
    Qangle = 24
    Qstrength = 3
    Qcoherence = 3
    patchsize = 5
    gradientsize = 9
    Q = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize, patchsize*patchsize))
    V = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize))
    mark = zeros((Qstrength*Qcoherence, Qangle, scale_factor*scale_factor))

    start_index = Int(ceil(patchsize/2))
    weights_matrix = rand(Normal(0, 2), 3*3)
    weights_matrix = Array(Diagonal(weights_matrix))
    colorview_rgb = channelview(YCbCr.(img))
    color_ve = colorview_rgb[1,:,:]
    colorview_rgb_gt = channelview(YCbCr.(ground_truth_im))
    color_ve_gt = colorview_rgb_gt[1,:,:]    
    w,h = size(color_ve)
    
    for row in start_index:Int(w)-start_index
        for col in start_index:Int(h)-start_index
                patch = color_ve[row-2:row+2,col-2:col+2]
                gradientpatch = color_ve[row-1:row+1,col-1:col+1]
                gx,gy = imgradients(patch,KernelFactors.ando3)
                gx = reshape(gx,patchsize*patchsize,1)
                gy = reshape(gy,patchsize*patchsize,1)
                GT = transpose([gx gy])

                GTWG = GT*weights_matrix*transpose(GT)

                eigen_max = eigmax(GTWG)
                eigen_min = eigmin(GTWG)
                eigen_vector_max = eigvecs(GTWG)[:,1]
                eigen_vector_min = eigvecs(GTWG)[:,2]
                gradient_angle = atan2(eigen_vector_max[2],eigen_vector_max[1])
                if gradient_angle < 0
                    gradient_angle = gradient_angle + pi
                end
                lamda = abs(sqrt(complex(eigen_max)))/Qstrength
                u =(sqrt(complex(eigen_max)) -  sqrt(complex(eigen_min)))/(sqrt(complex(eigen_max)) +  sqrt(complex(eigen_min)))/Qcoherence
                angle = floor(gradient_angle/pi*Qangle)

                if lamda < 0.0001
                    strength = 1
                elseif lamda > 0.001
                    strength = 3
                else
                    strength = 2
                end

                u = abs(u)
                if u < 0.25
                    coherence = 1
                elseif u > 0.5
                    coherence = 3
                else
                    coherence = 2
                end


                # Bound the output to the desired ranges
                if angle > 23
                    angle = 23
                elseif angle <= 0
                    angle = 1
                end
                angle = Int(angle)
                pixeltype = ((row-start_index) % scale_factor) * scale_factor + ((col-start_index) % scale_factor)
                pixelHR = color_ve_gt[row,col]
                
                # Compute A'A and A'b
                
                wp,hp = size(patch)
                patch_1 = reshape(patch,wp*hp,1)
                patch = reshape(patch,wp*hp)
                ATA = dot(transpose(patch_1),patch_1)
                ATb = patch*pixelHR
                
                Q[angle,strength,coherence,pixeltype+1,:,:] += ATA
                V[angle,strength,coherence,pixeltype+1,:] += ATb
          end
    end
    return [Q V]
end

In [8]:
GROUND_TRUTH_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/ground_truth/"
LOW_RES_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/low_res/"
INTERPOLATED_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/interpolated/"
addprocs(max(0, Sys.CPU_CORES-nprocs()))
filenames = readdir(INTERPOLATED_FOLDER)
all_pairs = []
for file in filenames
    if (contains("$file", ".jpg")) == true
        append!(all_pairs,(INTERPOLATED_FOLDER*file,GROUND_TRUTH_FOLDER*file))
    end
end

In [7]:
all_pairs

802-element Array{Any,1}:
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/.DS_Store" 
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/.DS_Store" 
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100007.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/100007.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100039.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/100039.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100075.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/100075.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100080.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/100080.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100098.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/ground_truth/100098.jpg"
 "/Users/manvithaponnapati/RAISR/train_data/interpolated/100099.jpg"
 ⋮                                                                  
 "/Users

In [10]:
# for np in 2:nprocs()
#     img,ground = pop!(all_pairs)
#     r=[@spawnat p compute_filters(img,ground) for p in procs()[1:np]]
#     s=fetch(r) 
# end

In [ ]:
#Log times

Average Epoch : 15 seconds for 480x520 pixel image
One core : 51480 seconds
3000 images
    
4 cores #Ec2 instance - Amazon Linux AMI
Average Epoch : 15 seconds for 480x520 pixel image
4 core : 17164.32234 seconds - 
3000 images